In [7]:
import io, os, sys, types
import graphlab as gl
from datetime import datetime
from os import path
import numpy as np
import import_ipynb
from decimal import *
import Creacion_Vectores as C_V
import Normalizacion as Norm
from collections import OrderedDict
import operator
import Conn_E_SQL as Conn


class content_based_recommender:
    
    def recommender_calculation (self, apartment_file_normalized,client_file_processed,Vector_IDF,client_number):
        
        #Creacion de Sframes
        
        apartment_temp_recom_matrix=gl.SFrame(apartment_file_normalized)
        client_temp_recom_matrix=gl.SFrame(client_file_processed)
                
        #Variables
        
        i=0
        m=0
        r=0
        v=0
        vector_size=7
        Array_Vector_IDF=np.asanyarray(Vector_IDF)
        single_client_recomm_scores=[]
        recommender_score=0
        values_apartment=[]
        values_client=[]
        apartment_recommender_score=[]
        
        #Tratamiento de Columnas
        apartment_temp_recom_matrix.remove_column('ID_V')
        apartment_temp_recom_matrix.remove_column('Zona')
        apartment_temp_recom_matrix.remove_column('Cuartos')
        apartment_temp_recom_matrix.remove_column('Cuota_Inicial')
        apartment_temp_recom_matrix.remove_column('Cuota_Credito')
        client_temp_recom_matrix.remove_column('ID_C')
        client_temp_recom_matrix.remove_column('Zona')
        client_temp_recom_matrix.remove_column('Cuartos')
        client_temp_recom_matrix.remove_column('Cuota_Inicial')
        client_temp_recom_matrix.remove_column('Cuota_Credito')
        
        #Creacion de vectores Decimales para Apartamento             
        while(i<apartment_temp_recom_matrix.num_rows()):
            j=0
            k=0                  
            temporal_dict_apartment=[]            
            ord_temporal_dict_apartment=[]        
            temporal_dict_apartment=apartment_temp_recom_matrix[i]
            ord_temporal_dict_apartment=(sorted(temporal_dict_apartment.iteritems()))                     
            for j,k in ord_temporal_dict_apartment:
                values_apartment.append(round(Decimal(k),4))
            i=i+1
        
        #Creacion de Vectores Decimales para Cliente
               
        while(m<client_temp_recom_matrix.num_rows()):
            h=0
            l=0
            temporal_dict_client=[]
            ord_temporal_dict_client=[]
            temporal_dict_client=client_temp_recom_matrix[m]
            ord_temporal_dict_client=(sorted(temporal_dict_client.iteritems()))
            for h,l in ord_temporal_dict_client:
                values_client.append(round(Decimal(l),4))
            m=m+1 
        
        #Calculo de Puntaje de recomendacion para el cliente s por apartamento t
        
        while(r<len(values_client)):
            p=[]
            p_array=[]
            s=0
            u=0
            recommender_score=0            
            if(r==0):
                while(s< vector_size):
                    p.append(values_client[s])
                    s=s+1
            else:
                s=r
                while(s< (r+vector_size)):
                    p.append(values_client[s])
                    s=s+1
                  
            while(u < len(values_apartment)):
                q=[]
                q_array=[]
                t=0
                if(u==0):
                    while(t< vector_size):
                        q.append(values_apartment[t])
                        t=t+1
                else:
                    t=u
                    while(t< (u+vector_size)):
                        q.append(values_apartment[t])
                        t=t+1
                p_array=np.asanyarray(p)
                q_array=np.asanyarray(q)
                recommender_score=round(np.sum(p_array*q_array*Array_Vector_IDF),5)
                u=u+vector_size
                apartment_recommender_score.append(recommender_score)
            r=r+vector_size
            
        #Construccion de Vector por Cliente
        
        Number_apartments= apartment_temp_recom_matrix.num_rows()
        inferior_limit=(client_number-1)*Number_apartments
        superior_limit=((client_number*Number_apartments))
        for v in range(inferior_limit,superior_limit):
            single_client_recomm_scores.append(apartment_recommender_score[v])
        
        return single_client_recomm_scores 

    def recommender_matrix (self, recommender_calculation, client_file_processed, client_number, apartment_file_processed, apartment_file_normalized, 
                            zona_number,visualization):   
        
        
        #Creacion de Sframes
            
        recommender_matrix=gl.SFrame()
                
        #Creacion variables
        zona_cliente=0
        cuartos_cliente=0
        cuota_inicial_cliente=0
        cuota_credito_cliente=0
        cliente_ID= client_file_processed["ID_C"]
        cliente_Vector=[]
        zonas_clientes=client_file_processed["Zona"]
        cuartos_clientes_array= client_file_processed["Cuartos"]
        cuota_inicial_clientes= client_file_processed["Cuota_Inicial"]
        cuota_credito_clientes= client_file_processed["Cuota_Credito"]
        zona_apartamentos=apartment_file_processed["Zona"]
        cuartos_apartamentos=apartment_file_processed["Cuartos"]
        cuota_credito_apartamentos=apartment_file_processed["Cuota_Credito"]
        cuota_inicial_apartamentos=apartment_file_processed["Cuota_Inicial"]
               
        i=0
        j=0
        k=0
        l=0
        m=0
        n=0
        o=0
        p=0
        recommendation_zona=[]
        recommendation_cuartos=[]
        recommendation_cuota_credito=[]
        recommendation_cuota_inicial=[]
        recommender_score=[]
        recommender_score_visualization=[]
        factor_zona_igual=1
        factor_zona_diferente=0.8
        factor_zona_diferente_sin_visualizacion=-1000
        factor_cuarto_igual=1.5
        factor_cuarto_mayor=1
        factor_cuarto_menor=0.2
        factor_credito_mayor=-1
        factor_credito_menor=0.2
        factor_credito_target=1
        rango_target=0.8
        factor_cuota_mayor=-0.8
        factor_cuota_menor=0.8
        factor_de_ajuste_credito=1.25
        factor_de_ajuste_cuota_inicial=1.5
        suma_a=0
        suma_b=0
        
        #Agregar ID_C
        
        while (p< len(apartment_file_normalized["ID_V"])):
            cliente_Vector.append(cliente_ID[0])
            p=p+1
        
        recommender_matrix.add_column(gl.SArray(cliente_Vector), name='ID_C')
        
        #modificacion de Sframes
        
        recommender_matrix.add_column(apartment_file_normalized["ID_V"], name='ID_V')
        recommender_matrix.add_column(gl.SArray(recommender_calculation), name='Recommender_Features_Score')
        recommender_features_score=recommender_matrix["Recommender_Features_Score"]
        
        #Asignar Cuarto, Cuota Inicial y Cuota Credito - La asignacion tiene que hacerse de forma consecutiva
         
        for i in cliente_ID :
            if(i==client_number):
                zona_cliente=zonas_clientes[i-1]
                cuartos_cliente=cuartos_clientes_array[i-1]
                cuota_credito_cliente=cuota_credito_clientes[i-1]
                cuota_inicial_cliente=cuota_inicial_clientes[i-1]
        
        #Agregar Puntaje de Recomendacion por Zona
        
        if(visualization==1):
            for m in zona_apartamentos:
                if (m==zona_cliente):
                    recommendation_zona.append(factor_zona_igual)
                else:
                    recommendation_zona.append(factor_zona_diferente)
            recommendation_zona=np.multiply(recommendation_zona,recommender_features_score)
            recommender_matrix.add_column(gl.SArray(recommendation_zona), name='Recommender_Zona')
        else:
            for n in zona_apartamentos:
                if (n==zona_cliente):
                    recommendation_zona.append(factor_zona_igual)
                else:
                    recommendation_zona.append(factor_zona_diferente_sin_visualizacion)
            recommendation_zona=np.multiply(recommendation_zona,recommender_features_score)
            recommender_matrix.add_column(gl.SArray(recommendation_zona), name='Recommender_Zona')
        
        
        #Agregar Puntaje de Recomendacion por Cuartos
        
        for j in cuartos_apartamentos:
            if (j==cuartos_cliente):
                recommendation_cuartos.append(factor_cuarto_igual)
            if (j>cuartos_cliente):
                recommendation_cuartos.append(factor_cuarto_mayor)
            if (j<cuartos_cliente):
                recommendation_cuartos.append(factor_cuarto_menor)
        
        recommendation_cuartos=np.multiply(recommendation_cuartos,recommender_features_score)
        recommender_matrix.add_column(gl.SArray(recommendation_cuartos), name='Recommender_Cuartos')
        
        #Agregar Puntaje Cuota de Credito
               
        for k in cuota_credito_apartamentos:
            if ((k/cuota_credito_cliente*float(factor_de_ajuste_credito))> 1):
                recommendation_cuota_credito.append(factor_credito_mayor)
            if rango_target < (k/cuota_credito_cliente*float(factor_de_ajuste_credito))< 1 :
                recommendation_cuota_credito.append(factor_credito_target)
            if ((k/cuota_credito_cliente*float(factor_de_ajuste_credito))<  rango_target):
                recommendation_cuota_credito.append(factor_credito_menor) 
            
        recommendation_cuota_credito=np.multiply(recommendation_cuota_credito,recommender_features_score)
        recommender_matrix.add_column(gl.SArray(recommendation_cuota_credito), name='Recommender_Cuota_Credito')
    
        #Agregar Puntaje Cuota Inicial   
        
        for l in cuota_inicial_apartamentos:
            if (l>cuota_inicial_cliente*float(factor_de_ajuste_cuota_inicial)):
                recommendation_cuota_inicial.append(factor_cuota_mayor)
            if (l<cuota_inicial_cliente*float(factor_de_ajuste_cuota_inicial)):
                recommendation_cuota_inicial.append(factor_cuota_menor)
        
        recommendation_cuota_inicial=np.multiply(recommendation_cuota_inicial,recommender_features_score)
        recommender_matrix.add_column(gl.SArray(recommendation_cuota_inicial), name='Recommender_Cuota_Inicial')
        
        #Calculo de Valor final de Recomendacion
                      
       
        for x in zip(np.asanyarray(recommender_features_score),recommendation_zona, recommendation_cuartos,recommendation_cuota_credito,recommendation_cuota_inicial):
                recommender_score.append(sum(x))
        
        recommender_matrix.add_column(gl.SArray(recommender_score), name='Recommender_Score')        
              
        recommender_matrix=recommender_matrix[recommender_matrix['Recommender_Score'] > 0]
                      
        recommender_matrix=recommender_matrix.sort('Recommender_Score', ascending = False)
        
        recommender_matrix.remove_columns(['Recommender_Features_Score', 'Recommender_Zona', 'Recommender_Cuartos', 'Recommender_Cuota_Credito', 
                                           'Recommender_Cuota_Inicial'  ])
        return recommender_matrix.topk('Recommender_Score', k=20)
  



Quieres ver apartamentos fuera de tu zona de escogencia?
1


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.027925 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,long,long,long,long,long,long,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.030917 secs.

+------+------+-------------------+
| ID_C | ID_V | Recommender_Score |
+------+------+-------------------+
|  1   | 350  |      0.047012     |
|  1   | 149  |      0.045543     |
|  1   | 142  |      0.045543     |
|  1   | 197  |      0.045288     |
|  1   | 198  |      0.045288     |
|  1   | 213  |      0.044931     |
|  1   | 214  |      0.044931     |
|  1   | 127  |      0.044344     |
|  1   | 126  |      0.044344     |
|  1   | 169  |      0.043884     |
+------+------+-------------------+
[20 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
